## Project Dublin Bus 


## Change the data format, remove duplication and  save it 

## Import package and data 

In [72]:
#Import the required packages
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

# Import package matplotlib  for visualisation/plotting
import matplotlib.pyplot as plt

#import matplotlib.patches as mpatches
import seaborn as sns

#import package matplotlib for import plots to PDF
from matplotlib.backends.backend_pdf import PdfPages

#For showing plots directly in the notebook
#%matplotlib inline

# Reading from a csv file, into a data frame
import numba

import time 
begin_time=time.time()

In [73]:
#Change the columns's names 
Columns_name = ['Timestamp','Line_ID','Direction','Journey_Pattern_ID','Date','Vehicle_Journey_ID','Operator',
              'Congestion','Lon_WGS84','Lat_WGS84','Delay','Block_ID','Vehicle_ID','Stop_ID','At_Stop']

In [74]:
#This is the data of January 2013
df = pd.read_csv('dublin_2012_week1_before.csv',names=Columns_name, dtype={ 'Journey_Pattern_ID': object} )
df.head(3)

/Users/April/anaconda/envs/comp47350/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
0,1352160000000000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0
1,1352160000000000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0
2,1352160000000000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0


In [75]:
# Check how many rows we have after loaded the csv file 
df.shape

(10856935, 15)

#  Data Cleaning 


In [76]:
#Drop duplicate rows 
df=df.drop_duplicates()
#Drop constant column
#df.drop(['Direction','Operator'],axis=1,inplace=True)
#Drop Unused column
df.drop(['Line_ID','Direction','Operator','Congestion','Delay','Block_ID','Vehicle_ID',],axis=1,inplace=True)

In [77]:
# Check how many rows we have after remove duplication
df.shape

(10856935, 8)

## Remove all Journey_Pattern_ID is null

In [78]:
# Drop all Journey_Pattern_ID is null 
df=df.drop(df.index[df['Journey_Pattern_ID'].isnull()])

In [79]:
#Check how many rows we deleted(Before it is (10856935, 15) )
df.shape

(10856077, 8)

In [80]:
# Drop all Journey_Pattern_ID is null 
df=df.drop(df.index[df['Journey_Pattern_ID'] == 'null'])

In [81]:
#Check how many rows we deleted(Before it is (10856935, 15) )
df.shape

(9211192, 8)

In [82]:
#Print all the unique Journey_Pattern_ID
#print("Unique Journey_Pattern_ID ", pd.unique(df.Journey_Pattern_ID.ravel()))

## Remove some noise data ( Like only have less than 13 rows in one trip  )
## Those data are not actually a full run 

In [83]:
df_groupby=df.groupby(['Vehicle_Journey_ID','Date','Journey_Pattern_ID'])

#size = df_groupby.size()
#size=size[size<13]

#keys=df_groupby.groups

# Filter all the data only have less than 13 rows data 
df_short_Journ=df_groupby.filter(lambda x: len(x) < 13 )

In [84]:
# Drop all the Journery that are less than a number 

df=pd.concat([df, df_short_Journ]).drop_duplicates(keep=False)

In [85]:
#Check how many rows we deleted
df.shape


(9172588, 8)

# Find the first stop, and remove all the error data before it 

In [86]:
# Groupby and get the first stop for every trip 
df_groupby=df.groupby(['Journey_Pattern_ID','Vehicle_Journey_ID','Date'])
df_first_stop=df_groupby.first()
df_last_stop=df_groupby.last()

In [87]:
# Groupby and get the first stop for every trip 
#df_groupby=df_all_jer.groupby(['Journey_Pattern_ID','Vehicle_Journey_ID','Date'])
#df_groupby.last()

In [88]:
# df_first_stop.index.levels[0] 
#df_first_stop.to_csv('Fist_stop.csv')

# Find the most frequent first stop 
df_first_stopid=df_first_stop.groupby(level=0)['Stop_ID'].agg(lambda x: x.value_counts().index[0])
df_last_stopid=df_last_stop.groupby(level=0)['Stop_ID'].agg(lambda x: x.value_counts().index[0])

In [89]:
df_new_first=df_first_stopid.to_frame()
df_new_last=df_last_stopid.to_frame()

In [90]:
df_new_first.head()

,Stop_ID
Journey_Pattern_ID,
00010001,226
00010002,265
00010003,226
00011001,381
00011002,381


In [91]:
df_new_last.head()

,Stop_ID
Journey_Pattern_ID,
00010001,380
00010002,380
00010003,271
00011001,226
00011002,10


In [92]:
#Add new index and Journey_Pattern_ID column
df_new_first.reset_index(inplace=True)
df_new_last.reset_index(inplace=True)

In [93]:
df_new_first = df_new_first.rename(columns={'Stop_ID': 'First_Stop'})
df_new_last = df_new_last.rename(columns={'Stop_ID': 'Last_Stop'})

In [94]:
# Add the new feature First_Stop to dataframe 
df = pd.merge(df, df_new_first,  how='left', left_on=['Journey_Pattern_ID'], right_on = ['Journey_Pattern_ID'])
df = pd.merge(df, df_new_last,  how='left', left_on=['Journey_Pattern_ID'], right_on = ['Journey_Pattern_ID'])

In [95]:
df.shape

(9172588, 10)

In [96]:
@numba.jit
def clean_first_last(df):
# Write the function to check the first stop and last stop

    df_grouped = df.groupby(['Journey_Pattern_ID','Date','Vehicle_Journey_ID'])
    vj_groups = []
    for jer_name, jer_group in df_grouped:
    #print(jer_group)
        first_stop=False
        last_stop=False
        for index, row in jer_group.iterrows():
            #print(row['Stop_ID'],row['First_Stop'])
            if not first_stop and row['Stop_ID']!=row['First_Stop']:
                jer_group.set_value(index, 'Stop_ID',  np.nan)
                row['Stop_ID']= np.nan
                #print('after change ',row['Stop_ID'],row['First_Stop'])
            elif not first_stop and row['Stop_ID']==row['First_Stop']:
                first_stop=True
            elif not last_stop and row['Stop_ID']!=row['Last_Stop']:
                continue
            elif not last_stop and row['Stop_ID']==row['Last_Stop']:
                last_stop=True
            else: 
                jer_group.set_value(index, 'Stop_ID',  np.nan)
                row['Stop_ID']= np.nan
                #jer_group.shape
        if first_stop and last_stop:       
            vj_groups.append(jer_group )
    return pd.concat(vj_groups)
df=clean_first_last(df)

CPU times: user 18min 11s, sys: 25.6 s, total: 18min 37s
Wall time: 50min 41s


In [97]:
df.shape # (1058747, 10)

(6512909, 10)

In [98]:
# Drop all stop_id is null
df.drop(df.index[df['Stop_ID'].isnull()],inplace=True)

# Drop all Journey_Pattern_ID is null 
#df=df.drop(df.index[df['Journey_Pattern_ID'].isnull()])

In [67]:
df.shape

(5705865, 10)

In [29]:
# Change Timestamp to a better way 
df['Timestamp'] = df['Timestamp'] //1000000
df.head(3)

,Timestamp,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Lon_WGS84,Lat_WGS84,Stop_ID,At_Stop,First_Stop
382152,1352,00010001,2012-11-06,5425,-6.26195,53.391151,226,1,226
382746,1352,00010001,2012-11-06,5425,-6.26195,53.391151,226,1,226
383251,1352,00010001,2012-11-06,5425,-6.26195,53.391151,226,1,226


# Add Distance feature 

In [ ]:
# Only keep the first row for every trip at the same Stop(keep both at or not_at stop)
df = df.drop_duplicates(['Vehicle_Journey_ID','Journey_Pattern_ID','Date','Lon_WGS84','Lat_WGS84'],keep='last')

In [ ]:
from geopy.distance import distance

def coor():
    prev = yield
    running_dist = 0
    
    while True:
        next_ = yield running_dist
        one_dist=distance(prev, next_).meters
        if one_dist>1000:
            running_dist += 100
        else:
            running_dist += one_dist
        #print("The running_dist is ",running_dist)
        prev = next_

        
def add_distance_all_runs(df):
    """
    :param df: dataframe before distance has been added
    :return: dataframe with distances for each run
    """
    df['Distance'] = -1
    vj_groups = []
    df_grouped = df.groupby(['Journey_Pattern_ID','Date','Vehicle_Journey_ID'])
    for v_name, v_group in df_grouped:
        c = coor()
        next(c)
            #print("A new loop")
        for index, row in v_group.iterrows():
            dist = c.send((row['Lat_WGS84'], row['Lon_WGS84']))   
            v_group.set_value(index, 'Distance', dist)
        if v_group['Distance'].max() < 36000:
            vj_groups.append(v_group)
    
    return pd.concat(vj_groups)


df=add_distance_all_runs(df)

# For some run, the trip is too long, then we could remove all the row after three and half hours. As some bus didn't stop after finish one trip 

In [ ]:
# Only keep the first row for every trip at the same Stop(keep both at or not_at stop)
df = df.drop_duplicates(['Vehicle_Journey_ID','Journey_Pattern_ID','Date','Stop_ID','At_Stop'],keep='last')

In [ ]:
# Add new feature Trip_Time 
zscore=lambda x: x - x.min()
df['Trip_Time']= df.groupby(['Vehicle_Journey_ID','Journey_Pattern_ID','Date'])['Timestamp'].transform(zscore)

In [ ]:
# Drop all the rows' Trip_Time is longer than four hours 
df=df.drop(df.index[df['Trip_Time']>=14400])

# Add some other features 

In [6]:
#Convert Timestamp to datetime and new column datetime to data frame 
df['datetime'] = pd.to_datetime(df['Timestamp'],unit='s')
df.head(5)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,datetime
0,1352160000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0,2012-11-06 00:00:00
1,1352160000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0,2012-11-06 00:00:00
2,1352160000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0,2012-11-06 00:00:00
3,1352160002,41,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1,2012-11-06 00:00:02
4,1352160002,63,0,null,2012-11-05,1116,D2,0,-6.171050,53.259201,292,63003,33137,3283,0,2012-11-06 00:00:02


In [7]:
# Add hourOfDay to dataframe
df['datetime']=pd.to_datetime(df['datetime'])
df['HourOfDay']=df['datetime'].dt.hour

In [8]:
#ADD day of the week to the date frame 
df['day_of_week'] = df['datetime'].dt.weekday_name
df.head(3)

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,datetime,HourOfDay,day_of_week
0,1352160000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0,2012-11-06,0,Tuesday
1,1352160000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0,2012-11-06,0,Tuesday
2,1352160000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0,2012-11-06,0,Tuesday


In [ ]:
# Add midweek feature 
df['midweek'] = df['day_of_week'].map({'Monday': 0, 'Tuesday': 0,'Wednesday':0,'Thursday':0,'Friday': 0,'Saturday':1,'Sunday':1
                                          })

# Add weather data 

In [9]:
def add_time_bin_column(x):
    if x <= 4:
        return 'early_am'
    elif x >= 5 and x <= 12:
        return 'am'
    elif x >= 12 and x <= 20:
        return 'pm'
    elif x >= 21:
        return 'late_pm'

df['time_bin']=df['HourOfDay'].map(lambda x:add_time_bin_column(x))
# Add midweek feature 

#s.map(lambda x: 'this is a string {}'.format(x),na_action='ignore')


In [10]:
df.head()

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Date,Vehicle_Journey_ID,Operator,Congestion,Lon_WGS84,Lat_WGS84,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,datetime,HourOfDay,day_of_week,time_bin
0,1352160000,15,0,00150001,2012-11-05,5826,RD,0,-6.258584,53.340099,-361,15013,33210,4870,0,2012-11-06 00:00:00,0,Tuesday,early_am
1,1352160000,46,0,046A1002,2012-11-05,7267,D2,0,-6.259093,53.345425,-1101,46004,36024,794,0,2012-11-06 00:00:00,0,Tuesday,early_am
2,1352160000,14,0,00140001,2012-11-05,6206,D2,0,-6.257329,53.287521,-126,14003,33325,1047,0,2012-11-06 00:00:00,0,Tuesday,early_am
3,1352160002,41,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1,2012-11-06 00:00:02,0,Tuesday,early_am
4,1352160002,63,0,null,2012-11-05,1116,D2,0,-6.171050,53.259201,292,63003,33137,3283,0,2012-11-06 00:00:02,0,Tuesday,early_am


In [15]:
df_weather=pd.read_csv('weather_output.csv')

In [16]:
df_weather.head()

,Date,time_bin,cloud,rain,temp,wind
0,2012-11-01,early_am,3.60,0.00,1.30,7.00
1,2012-11-01,am,5.75,0.00,1.70,12.50
2,2012-11-01,pm,3.88,0.12,5.29,17.25
3,2012-11-01,late_pm,2.67,0.00,4.73,19.67
4,2012-11-02,early_am,6.00,0.00,4.66,16.80


In [17]:
df_weather.dtypes

Date         object
time_bin     object
cloud       float64
rain        float64
temp        float64
wind        float64
dtype: object

In [18]:
df.shape

(1765912, 19)

In [19]:
df = pd.merge(df, df_weather,  how='left', left_on=['Date','time_bin'], right_on = ['Date','time_bin'])

In [20]:
df.shape

(1765912, 23)

# Save the file for modeling 

In [27]:
#Save the data for next step
df.to_csv('dublin_2012_week1_distance.csv',index=False)

In [ ]:
End_time=time.time()
Duringtime=End_time-begin_time
print("The time used is ",Duringtime//60)